# Get game list

In [16]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os.path
import json
import datetime
n=404
def getList(n):
    linklist=[]
    IDlist=[]
    for page in range(n):
        soup = BeautifulSoup(requests.get('https://store.steampowered.com/search/?ignore_preferences=1&category1=998&os=win&filter=globaltopsellers&page=%d'%page).text)
        soups= soup.find_all(href=re.compile(r"https://store.steampowered.com/app/"))
        for i in soups:
            i = i.attrs['href']
            res=re.search('https://store.steampowered.com/app/(\d+)/',i)
            linklist.append(res.group(0))
            IDlist.append(res.group(1))
    return linklist,IDlist
if os.path.isfile('gamelist.csv'): #cache gamelist
    df = pd.read_csv('gamelist.csv')
else:
    linklist,IDlist=getList(n)
    df = pd.DataFrame(list(zip(linklist,IDlist)), columns =['Link', 'ID'])
    df.to_csv('gamelist.csv')

# Get game data

In [17]:
def getPrice(soup):
    originalPrices = soup.select(".game_area_purchase_game .game_purchase_action .game_purchase_price")
    if len(originalPrices)>0:
        for i in range(len(originalPrices)):
            if re.search('\$',originalPrices[i].string):
                originalPrice = float(re.search('\$([\d.]+)',originalPrices[i].string).group(1))
                return originalPrice, originalPrice
            elif re.search('Free',originalPrices[i].string, re.IGNORECASE):
                return 0.,0.
    originalPrices = soup.select(".game_area_purchase_game .game_purchase_action .discount_original_price")
    discountPrices = soup.select(".game_area_purchase_game .game_purchase_action .discount_final_price")
    for i in range(len(originalPrices)):
        if re.search('\$',originalPrices[i].string):
            originalPrice = float(re.search('\$([\d.]+)',originalPrices[i].string).group(1))
            discountPrice = float(re.search('\$([\d.]+)',discountPrices[i].string).group(1))
            return originalPrice, discountPrice
        elif re.search('Free',originalPrices[i].string, re.IGNORECASE):
            return 0.,0.

def getReview(soup):
    try:
        review = str.strip(str(soup.find_all(class_="nonresponsive_hidden responsive_reviewdesc")[-1].contents[0]))
        rating = re.search('(\d+)\% of the ([\d,]+) user', review).group(1)
        rating = int(rating)
        reviewNum = re.search('(\d+)\% of the ([\d,]+) user', review).group(2)
        reviewNum = int(reviewNum.replace(',',''))
        return rating, reviewNum
    except:
        return None, 0

if not os.path.isfile('gameData.json'):
    gameFile = open('gameData.json','w')
    gameData = {}
    gameFile.close()
else:
    gameFile = open('gameData.json','r')
    try:
        gameData = json.load(gameFile)
    except:
        print('empty file')
        gameData = {}
    gameFile.close()
for p in range(len(df)):
    if p%100==0:
        print(p)
    if str(df['ID'][p]) not in gameData.keys():
        soup = BeautifulSoup(requests.get(df['Link'][p]).text)
        name = str(soup.find(class_="apphub_AppName").string)
        try:
            date = str(soup.find(class_="date").string)
        except:
            date = None
        try:
            originalPrice, discountPrice = getPrice(soup)
        except:
            originalPrice, discountPrice = 0.,0.
        rating, reviewNum = getReview(soup)
        link = df['Link'][p]
        description = str.strip(str(soup.find(class_="game_description_snippet").string))
        gameData[str(df['ID'][p])] = [name,date,originalPrice,discountPrice,rating,reviewNum,link,description]
gameFile = open('gameData.json','w')
gameFile.write(json.dumps(gameData))
gameFile.close()

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500


# Save data into tree

In [18]:
# class Node:
#     def __init__(self, datatuple=None):
#         if datatuple is not None:
#             self.id = datatuple[0]
#             self.name = datatuple[1][0]
#             self.date = pd.to_datetime(datatuple[1][1])
#             if self.date is None:
#                 self.date = pd.to_datetime('2077-01-01')
#             self.oriPrice = datatuple[1][2]
#             self.discPrice = datatuple[1][3]
#             self.rating = datatuple[1][4]
#             if self.rating is None:
#                 self.rating = 0
#             self.reviews = datatuple[1][5]
#             self.link = datatuple[1][6]
#             self.desc = datatuple[1][7]
#             self.indices = [self.discPrice, self.date, self.rating]
#         self.left = None
#         self.right = None

#     def disp(self):
#         print(self.id,self.name,self.date,self.oriPrice,self.discPrice,self.rating,self.reviews,self.link,self.desc)
    
#     def dispTree(self):
#         if self.left:
#             self.left.dispTree()
#         self.disp()
#         if self.right:
#             self.right.dispTree()
    
# def constructTree(nodeList, depth=0):
#     try:
#         axis = depth % len(nodeList[0].indices)
#     except: #list empty
#         return None
#     nodeList.sort(key = lambda x: x.indices[axis])
#     median = len(nodeList)//2
#     medianNode = nodeList[median]
#     medianNode.left = constructTree(nodeList[:median], depth+1)
#     medianNode.right = constructTree(nodeList[median+1:], depth+1)
#     return medianNode

# def rangesearch(tree, bounds=[[0,999], [pd.to_datetime('1970-01-01'), pd.to_datetime('2023-01-01')], [0,100]], depth=0):
#     res = []
#     try:
#         axis = depth % len(tree.indices)
#     except: #tree empty
#         return res
#     if tree.indices[0]>=bounds[0][0] and tree.indices[0]<=bounds[0][1] \
#     and tree.indices[1]>=bounds[1][0] and tree.indices[1]<=bounds[1][1]\
#     and tree.indices[2]>=bounds[2][0] and tree.indices[2]<=bounds[2][1]:
#         res.append(tree)
#         # res[0].desc=[lowerBound[axis],upperBound[axis]]
#         if tree.left is None and tree.right is None:
#             return res
#     if tree.indices[axis]<bounds[axis][1]:
#         res += rangesearch(tree.right, bounds, depth+1)
#     if tree.indices[axis]>bounds[axis][0]:
#         res += rangesearch(tree.left, bounds, depth+1)
#     return res

# nodeList = []
# for a in gameData.items():
#     nodeList.append(Node(a))
# tree = constructTree(nodeList)



# Range search

In [19]:
# reslist=rangesearch(tree, [[10,20], [pd.to_datetime('2022-01-01'), pd.to_datetime('2023-01-01')],[90,100]])
# for res in reslist:
#     # if res.discPrice>20 or res.discPrice<10 or res.rating<90:
#     res.disp()